In [100]:
import sys
sys.path.append("..")
from dotenv import load_dotenv
load_dotenv()

import omeka_extractor as oe

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [101]:
raw_items = oe.fetch_all_items()

Retrieved 740 items.


In [ ]:
items = [oe.filter_json(item) for item in raw_items]

`filter json files`

In [92]:
filtered_json = []
for item in raw_items:
    filtered_json.append({
        "id": item["id"],
        "item_type__id": item.get("item_type").get("id"),
        "item_type__name": item.get("item_type").get("name"),
        "files__count": item.get("files").get("count"),
        "tags": item.get("tags"),
        "element_texts": item.get("element_texts"),
    })

df = pd.json_normalize(filtered_json, sep="__")

`extract tags df`

In [ ]:
tags_columns = [
    'id',
    'item_type__id', 'item_type__name',
    'files__count',
    'tags__id', 'tags__name'
]
df_tags = pd.json_normalize(
    filtered_json,
    record_path='tags',
    meta=[
        'id',
        'item_type__id',
        'item_type__name',
        'files__count'
    ],
    sep="__",
    record_prefix='tags__'
)
df_tags = df_tags[tags_columns]

`extract metadata df`

In [99]:
element_columns = [
    'id',
    'item_type__id', 'item_type__name',
    'files__count',
    'element_set__id', 'element_set__name',
    'element__id', 'element__name',
    'text'
]
df_elements = pd.json_normalize(
    filtered_json,
    record_path='element_texts',
    meta=['id','item_type__id','item_type__name','files__count'],  # optionally keep item ID
    sep="__"
)

df_elements = df_elements[element_columns]